하루치 뉴스를 분석할 수도 있지만, 때로는 미리 분석하려는 모든 날의 뉴스들로부터 단어사전을 만들어둘 필요도 있습니다. 이번에는 corpus_10days의 10일치 뉴스에 대해서 명사를 모두 추출한 뒤, universial dictionary를 만드는 것을 알아보고, 이를 이용하여 2016-10-20 뉴스에 대하여 뉴스의 군집화를 한 뒤, 각 군집의 키워드를 추출함으로써 그날 뉴스의 핫키워드를 추출해 보도록 하겠습니다. 

Day5_2에서는 doc2vec을 이용하였기 때문에 각 영화에 해당하는 corpus를 다시 loading 하여야 했습니다. 이번에는 각 뉴스 문서 별로 term frequency matrix를 만들고, 이를 이용하여 곧바로 Regularized Logistic Regression을 통하여 키워드를 선택하겠습니다. 

명사 셋을 추출할 것인지에 대한 configuration parameters를 아래에 적어뒀습니다. 

In [99]:
EXTRACT_NOUN_SET = False
noun_dictionary_fname = '../../../data/corpus_10days/models/noun_dictionary.txt'

TOKENIZE_2016_1020 = False
normed_corpus_fname = '../../../data/corpus_10days/news/2016-10-20_article_all_normed.txt'
tokenized_corpus_fname = '../../../data/corpus_10days/news/2016-10-20_article_all_normed_nountokenized.txt'

corpus_10days의 10일치 뉴스에 대하여, 각 날의 기사마다 명사를 추출하여, 이를 nouns_dictionary에 누적하여 저장하겠습니다. 

import는 if 구문 아래에서 실행할 수도 있습니다. 이 때에는 EXTRACT_NOUN_SET = True 일 때에만, glob, sys, corpus 등이 import 됩니다. 

glob는 조건을 만족하는 파일 주소들을 찾을 수 있는 library입니다. 

    corpus_fnames = glob.glob('../../../data/corpus_10days/news/*_article_all_normed.txt')
    
위 구문은 root/data/corpus_10days/news 폴더에 있는 파일들 중에서 앞 부분이 어떤 것들이 등장하던지, 뒷부분에 아래의 글자가 있는 파일들을 찾으라는 의미입니다. 

    _article_all_normed.txt
    
그 결과는 아래와 같습니다. 

    ['../../../data/corpus_10days/news/2016-10-28_article_all_normed.txt',
     '../../../data/corpus_10days/news/2016-10-26_article_all_normed.txt',
     '../../../data/corpus_10days/news/2016-10-22_article_all_normed.txt',
     '../../../data/corpus_10days/news/2016-10-21_article_all_normed.txt',
     '../../../data/corpus_10days/news/2016-10-29_article_all_normed.txt',
     '../../../data/corpus_10days/news/2016-10-23_article_all_normed.txt',
     '../../../data/corpus_10days/news/2016-10-20_article_all_normed.txt',
     '../../../data/corpus_10days/news/2016-10-25_article_all_normed.txt',
     '../../../data/corpus_10days/news/2016-10-24_article_all_normed.txt',
     '../../../data/corpus_10days/news/2016-10-27_article_all_normed.txt']
     
"for corpus_fname in corpus_fnames"의 for loop을 돌면서, 매일의 뉴스 기사마다 명사를 추출합니다. 
nouns_dictionary는 set이기 때문에 update를 이용할 수 있습니다. 

    print('corpus name = %s, num nouns = %d' % (corpus_name, len(nouns_dictionary)))
    
아래의 구문을 통하여 하루의 뉴스기사를 분석할 때마다, 명사사전의 크기가 얼마나 커져가는지도 확인할 수 있습니다. 

이번에는 피클링을 하지 않고 명사 사전을 텍스트 파일로 저장하겠습니다. 피클링은 데이터를 binary로 저장하기 때문에 파일로 직접 읽을 수가 없습니다. 하지만 텍스트 파일로 저장한다면 눈으로 확인할 수 있을 뿐더러, 다른 프로그래밍 언어로 읽을 수도 있습니다. 



In [30]:
if EXTRACT_NOUN_SET:
    import glob
    import sys
    sys.path.append('../soy/')
    sys.path.append('../mypy')
    from soy.nlp.tags import LRNounExtractor
    from corpus import Corpus

    corpus_fnames = glob.glob('../../../data/corpus_10days/news/*_article_all_normed.txt')
    print('num corpus = %d' % len(corpus_fnames))
    
    nouns_dictionary = set()
    
    for corpus_fname in corpus_fnames:
        news_corpus = Corpus(corpus_fname, iter_sent=True)
        noun_extractor = LRNounExtractor()
        nouns, cohesion_probability = noun_extractor.extract(news_corpus)
        
        nouns_dictionary.update(set(nouns.keys()))
        corpus_name = corpus_fname.split('/')[-1].split(')')[0]
        print('corpus name = %s, num nouns = %d' % (corpus_name, len(nouns_dictionary)))
        
    with open(noun_dictionary_fname, 'w', encoding='utf-8') as f:
        for noun in nouns_dictionary:
            f.write('%s\n' % noun)
    
    print('done')

2016-10-20일 뉴스에 대하여 CountVectorizer를 이용하여 term frequency matrix를 만든 뒤, 그날의 주요 뉴스들이 어떤 것들이 있었는지 군집화를 수행합니다. scikit learn의 KMeans는 sparse matrix로도 군집화를 수행할 수 있습니다. 

이를 위하여, 이전 수업 때 수행했던 custom_tokenizer를 이용합니다. noun_dictionary는 텍스트 파일로 저장되어 있으므로, 파일을 open으로 열 때에는 반드시 encoding='utf-8'을 신경써 주세요.

    [noun.strip() for noun in f]
    
위 구문에서 noun.strip()을 하면 줄바꿈 기호 '\n'이 사라집니다. 

In [41]:
with open(noun_dictionary_fname, encoding='utf-8') as f:
    nouns_dictionary = [noun.strip() for noun in f]
    print(len(nouns_dictionary))

def custom_tokenize(doc):    
    def parse_noun(token):
        for e in reversed(range(1, len(token)+1)):
            subword = token[:e]
            if subword in nouns_dictionary:
                return subword
        return ''
    
    nouns = [parse_noun(token) for token in doc.split()]
    nouns = [word for word in nouns if word]
    return nouns

custom_tokenize('이번 국정농단의 사태에 대하여 뉴스를 이용한 분석을 수행합니다')

45930


['이번', '국정농단', '사태', '대하여', '뉴스', '이용', '분석', '수행']

2016-10-20 뉴스에 대하여 corpus를 만듦니다. corpus의 length도 확인합니다. 당일에 30,091개의 뉴스가 있습니다. 

In [43]:
if TOKENIZE_2016_1020:
    
    import sys
    sys.path.append('../mypy')
    from corpus import Corpus
    corpus_2016_1020 = Corpus(normed_corpus_fname, iter_sent=False)
    print('corpus length of 2016-10-20 = %d' % len(corpus_2016_1020))

    tokenized_corpus_2016_1020 = []
    for num_doc, doc in enumerate(corpus_2016_1020):
        if num_doc % 100 == 0:
            sys.stdout.write('\rtokenizing %d ... ' % num_doc)
        doc = ' '.join([noun for sent in doc.split('  ') for noun in custom_tokenize(sent)]).strip()
        tokenized_corpus_2016_1020.append(doc)    
    print('\rtokenization was done')

    with open(tokenized_corpus_fname, 'w', encoding='utf-8') as f:
        for doc in tokenized_corpus_2016_1020:
            f.write('%s\n' % doc)
            
else:
    
    with open(tokenized_corpus_fname, encoding='utf-8') as f:
        tokenized_corpus_2016_1020 = [doc.strip() for doc in f]

tokenization was done


CountVectorizer를 이용하여 term frequency matrix를 만듦니다. 만들어 둔 term frequency matrix는 corpus_10days/models/ 아래에 저장해 두었습니다. 

In [44]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=0.005, max_df=0.8)
x_2016_1020 = vectorizer.fit_transform(tokenized_corpus_2016_1020)
print(x_2016_1020)

from scipy.io import mmwrite
mmwrite('../../../data/corpus_10days/models/2016-10-20_noun_x.mm', x_2016_1020)

<30091x2631 sparse matrix of type '<class 'numpy.int64'>'
	with 1470416 stored elements in Compressed Sparse Row format>

x_2016_1020의 각 column에 해당하는 단어를 decoding 하기 위하여 vectorizer.vocabulary_로부터  int2vocab를 만들었습니다. 

In [100]:
int2vocab = sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])
int2vocab = [word for word, idx in int2vocab]
print(int2vocab[:5])

['00', '000', '01', '02', '03']


만들어진 x_2016_1020을 이용하여 Spherical kmeans를 수행하겠습니다. 이를 위하여 L2로 normalization을 수행한 뒤, 클러스터의 개수를 1000개로 지정하여 k-means clustering을 수행합니다. 

군집화나 토픽 모델링을 수행할 때에는 예상하는 것보다 군집/토픽의 개수를 크게 잡아주세요. 중복되는 군집이 등장하면 나중에 묶으면 됩니다. 하지만, 데이터에 노이즈가 있어서 다른 군집들이 하나의 군집으로 묶인다면 나중에 해석하기가 어려워집니다. 

특히, 여러 군집/토픽에서 두루두루 사용될 수 있는 단어들이 이러한 노이즈 역할을 합니다. 이런 단어들을 미리 걸러낼 수 있다면 훨씬 더 정교한 모델링이 될 것입니다. (즉, 군집화나 토픽모델링에서는 불필요하다 생각되는 단어들을 과감히 쳐낼수록 결과가 깔끔합니다).  그렇지 않다면, 군집화/토픽모델링을 할 때 군집/토픽의 개수를 크게 잡아두면 좀 더 좋습니다. 

군집화를 수행한 뒤, 20개의 군집들에 대하여 각 군집에 할당된 뉴스의 개수가 몇 개인지 출력합니다. 

In [95]:
%%time

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from collections import defaultdict

x_2016_1020 = normalize(x_2016_1020, axis=1, norm='l2')
kmeans = KMeans(n_clusters=1000, max_iter=20, n_init=1, verbose=1)
clusters = kmeans.fit_predict(x_2016_1020)

clusters_to_rows = defaultdict(lambda: [])
for idx, label in enumerate(clusters):
    clusters_to_rows[label].append(idx)

for i in range(20):
    print('cluster # %d has %d docs' % (i, len(clusters_to_rows[i])))

Initialization complete
Iteration  0, inertia 16587.190
Iteration  1, inertia 13143.934
Iteration  2, inertia 12694.927
Iteration  3, inertia 12486.385
Iteration  4, inertia 12372.992
Iteration  5, inertia 12308.900
Iteration  6, inertia 12272.808
Iteration  7, inertia 12250.201
Iteration  8, inertia 12232.298
Iteration  9, inertia 12220.497
Iteration 10, inertia 12213.461
Iteration 11, inertia 12207.667
Iteration 12, inertia 12200.940
Iteration 13, inertia 12194.250
Iteration 14, inertia 12189.779
Iteration 15, inertia 12187.236
Iteration 16, inertia 12185.546
Iteration 17, inertia 12184.778
Iteration 18, inertia 12184.223
Iteration 19, inertia 12183.920
cluster # 0 has 1 docs
cluster # 1 has 48 docs
cluster # 2 has 18 docs
cluster # 3 has 290 docs
cluster # 4 has 34 docs
cluster # 5 has 1 docs
cluster # 6 has 19 docs
cluster # 7 has 2 docs
cluster # 8 has 2 docs
cluster # 9 has 891 docs
cluster # 10 has 13 docs
cluster # 11 has 20 docs
cluster # 12 has 19 docs
cluster # 13 has 1 docs

군집 15번은 71개의 뉴스가 묶여있습니다 (이건 군집화를 할 때마다 달라지므로, 다시 실행시키시면 다른 숫자일겁니다). 이 군집에 대하여 키워드를 추출하기 위해서는, 15번 군집에 해당하는 뉴스의 label을 1로, 다른 뉴스들을 -1로 둔 뒤, 해당 군집 15번을 구분할 수 있는 주요 단어들을 L1 regularized logistic regression을 이용하여 뽑을 수도 있습니다. 

y를 만든 뒤, 혹시 하는 마음에 x_2016_1020.shape과 같은지 확인도 해봅시다. 

In [85]:
cluster_aspect = 15
y = [1 if i in clusters_to_rows[cluster_aspect] else -1 for i in range(x_2016_1020.shape[0])]
len(y), x_2016_1020.shape

(30091, (30091, 2631))

LogisticRegression의 penalty를 'l1'으로 주고, Regularization cost를 1로 주었습니다. 키워드의 개수가 너무 많다면 C를 더 작게, 키워드의 개수가 너무 적다면 C를 좀 더 크게 조절할 수 있습니다. 

    상위 30개의 키워드: 상승 (18.446), 하락 (9.586), 미국 (9.543), 전날 (6.326), 상승세 (6.213), 예상 (6.110), 거래 (5.957), 달러 (5.282), 기록 (5.227), 증시 (5.121), 이날 (4.866), 지수 (3.715), 시장 (3.581), 마감 (3.168), 강세 (1.732), 힐러리 (1.280), 대비 (1.085), 금리 (1.021), 51 (0.270)
    
상위 30개의 키워드를 선택하였더니, 미국 금리에 의한 증시의 변동에 대한 이야기임을 추정할 수 있습니다. 

In [87]:
from sklearn.linear_model import LogisticRegression

logistic_l1 = LogisticRegression(penalty='l1', C=1)
logistic_l1.fit(x_2016_1020, y)

keywords = sorted(enumerate(logistic_l1.coef_[0]), key=lambda x:x[1], reverse=True)[:30]
for word, score in keywords:
    if score == 0: break
    print('%s (%.3f)' % (int2vocab[word], score))

상승 (18.446)
하락 (9.586)
미국 (9.543)
전날 (6.326)
상승세 (6.213)
예상 (6.110)
거래 (5.957)
달러 (5.282)
기록 (5.227)
증시 (5.121)
이날 (4.866)
지수 (3.715)
시장 (3.581)
마감 (3.168)
강세 (1.732)
힐러리 (1.280)
대비 (1.085)
금리 (1.021)
51 (0.270)


이번에는 관심있는 군집들에 대하여 키워드들을 뽑아내는 함수를 만들겠습니다. interested_clusters로 입력되는 클러슽들에 대하여 위처럼 각각의 클러스터에 대한 Regularized Logistic Regression을 이용하여 키워드를 추출합니다. 

Regularization cost와 키워드의 개수를 선택하기 위하여 print_keywords의 함수의 arguments에 이를 넣어둡니다. L1 Logistic Regression에서 coefficient가 0이라는 것은 classification에서 유의미한 변수가 아니라는 뜻입니다. 그렇기 때문에 아래의 구문을 넣어서 coefficient가 0보다 큰 단어들만을 선택합니다. 

    keywords = [int2vocab[word] for word, score in keywords if score > 0]

아래와 같은 결과를 얻을 수 있습니다. 단어 선택과 같은 튜닝 과정없이 명사 추출 + k-means clustering + 키워드 추출만으로도 어느 정도 그날 뉴스의 토픽들을 살펴볼 수 있습니다. 

이제 남은 일은, 좀 더 정확한 단어를 선택하여 군집화가 잘 되게 만들고, 키워드를 추출하기에 좋은 parameter를 찾는 것입니다. 

하지만 이런 접근 방법에 한 가지 단점이 있습니다. 만약 18번 군집에 대한 키워드로 '디자이너'가 선택되었지만, 이 단어는 다른 군집에서의 키워드가 될 수도 있습니다. 다른 여러개의 군집에서도 키워드로 이용될 수 있는 단어라면, 18번 군집의 키워드로 선택이 되지 않을수도 있습니다. 근본적인 원인은 18번이 아닌 다른 군집에 실제 18번 군집과 비슷한 군집이 있기 때문입니다. 이는 topic modeling, doc2vec, 단어 선택을 통한 군집화의 고도화 등으로 해결해야 합니다. 

cluster # 3 keywords (from 290 news)
 > ['사진', '뉴시스', '제공', '전주', '울산', '장관', '발생']

cluster # 15 keywords (from 71 news)
 > ['엑스포츠뉴스', '을지로', '오후']

cluster # 17 keywords (from 299 news)
 > ['서울', '사진', '뉴시스', '여의도', '대한', '촉구', '국회', '브리핑', '이종', '스포츠재단', '강남구', '영상', '전국', '판매', '미래창조과학부', '선보', '오전', '83']

cluster # 18 keywords (from 280 news)
 > ['유진', '디자이너', '진행', '브랜드', '마이데일리']

cluster # 21 keywords (from 78 news)
 > ['디자이너', '조망', '콜렉션', '펼쳐진다', '헤라', '오전', '서울패션위크']

cluster # 22 keywords (from 62 news)
 > ['콤비', '사랑', '코미디', '개봉', '리포트', '11월']

cluster # 32 keywords (from 74 news)
 > ['버튼', '감상', '크기', '09', '눈물', '뮤직비디오', '27', '방탄소년단', '이미지']

cluster # 35 keywords (from 97 news)
 > ['권리', '있습니다', '주제', '사회', '자유', '여성', '사랑', '함께', '인터넷', '합니다', '주인', '지역']

cluster # 36 keywords (from 55 news)
 > ['을지로', '전자신문', '오후', '취하고', '엔터온뉴스', '컬렉션', '서울패션위크']

cluster # 43 keywords (from 56 news)
> ['재단', '검찰', '설립', '조사', '수사', '고발', '최씨', '미르']

cluster # 57 keywords (from 59 news)
> ['발사', '실패', '지난', '추가', '미사일', '무수단']

cluster # 146 keywords (from 73 news)
> ['대통령', '박근혜', '최순실', '의장', '국민', '정부', '경북', '우병우']

cluster # 856 keywords (from 93 news)
> ['학교', '학생들', '학생', '활동', '지원', '청소년', '아이들']

cluster # 857 keywords (from 56 news)
> ['한미', '북한', '양국', '압박', '대북', '외교', '국방', '협의체', '확장억제', '대한', '신설']

cluster # 909 keywords (from 98 news)
> ['의원', '국회', '지적', '2014년', '경우', '문제', '이전', '제출', '세금', '정부']

cluster # 938 keywords (from 73 news)
> ['토론', '클린턴', '대선', '여론조사', '트럼프']

cluster # 959 keywords (from 141 news)
> ['방송', '시즌', '시청자들', '지금', '프로그램', '뷰티', '정도', '오후', '여성', '지난', '후문', '제작', '8시']

cluster # 971 keywords (from 101 news)
> ['기술', '사용', '적용', '소재', '병원', '디자인', '선보', '출시', '기존', '겨울', '착용', '스포츠']

cluster # 980 keywords (from 56 news)
> ['여의도', '뉴스1', '국민의당', '국회', '토론회', '모두발언', '서울', '대표', '오전']

In [97]:
def print_keywords(interested_clusters, x, int2vocab, clusters_to_rows, C=1, topn=30):
    
    for cluster_id in interested_clusters:
        interested_docs = clusters_to_rows[cluster_id]
        print('\ncluster # %d keywords (from %d news)' % (cluster_id, len(interested_docs)))
        y = [1 if i in interested_docs else -1 for i in range(x_2016_1020.shape[0])]
        
        logistic_l1 = LogisticRegression(penalty='l1', C=C)
        logistic_l1.fit(x, y)

        keywords = sorted(enumerate(logistic_l1.coef_[0]), key=lambda x:x[1], reverse=True)[:topn]
        keywords = [int2vocab[word] for word, score in keywords if score > 0]
        print(' > %s' % (keywords))
        
interested_clusters = [i for i, labels in clusters_to_rows.items() if 50 < len(labels) < 500]
print_keywords(interested_clusters, x_2016_1020, int2vocab, clusters_to_rows, C=1.0)


cluster # 3 keywords (from 290 news)
 > ['사진', '뉴시스', '제공', '전주', '울산', '장관', '발생']

cluster # 15 keywords (from 71 news)
 > ['엑스포츠뉴스', '을지로', '오후']

cluster # 17 keywords (from 299 news)
 > ['서울', '사진', '뉴시스', '여의도', '대한', '촉구', '국회', '브리핑', '이종', '스포츠재단', '강남구', '영상', '전국', '판매', '미래창조과학부', '선보', '오전', '83']

cluster # 18 keywords (from 280 news)
 > ['유진', '디자이너', '진행', '브랜드', '마이데일리']

cluster # 21 keywords (from 78 news)
 > ['디자이너', '조망', '콜렉션', '펼쳐진다', '헤라', '오전', '서울패션위크']

cluster # 22 keywords (from 62 news)
 > ['콤비', '사랑', '코미디', '개봉', '리포트', '11월']

cluster # 32 keywords (from 74 news)
 > ['버튼', '감상', '크기', '09', '눈물', '뮤직비디오', '27', '방탄소년단', '이미지']

cluster # 35 keywords (from 97 news)
 > ['권리', '있습니다', '주제', '사회', '자유', '여성', '사랑', '함께', '인터넷', '합니다', '주인', '지역']

cluster # 36 keywords (from 55 news)
 > ['을지로', '전자신문', '오후', '취하고', '엔터온뉴스', '컬렉션', '서울패션위크']

cluster # 41 keywords (from 74 news)
 > ['헤라서울패션위크', '김창', '스타뉴스', '컬렉션']

cluster # 43 keywords (from 56 news)
 > [